# 商家相关特征

## 基本统计
#### m1,单个商家的消费总数
- 根据不同商家 id 累加消费记录


#### m2,单个商家的优惠券使用数量
- 根据不同商家 id 累加消费券使用记录

#### m3,单个商家发出多少优惠券
- 根据不同商家 id 累加消费券数量


#### m4-5678, 单个商家核销消费券的用户到该商家的距离（平均数/极大值/极小值/中位数）
- 根据商家 id 统计消费者距离，并计算基本统计量。
- 用以观测商家对核销优惠券的影响范围。

## 特征结合
#### m8，单个商家优惠券核销率
- 核销率 = 消费券使用量（M2）/消费券发行量（M3）。
- 反映发出的消费券在该商家的使用率。侧重于商家发行消费券。
- 核销率越高，则表明该商家发行的消费券越有效用户更可能在使用消费券

#### m9，单个商家销售中使用优惠券的概率
- 概率 = 消费券使用量（M2）/总消费量（M1），反映用户在该商家使用消费券的比率
- 概率越大，用户更可能在该商家消费时使用优惠券。
- 侧重于用户在该商家的消费。

### 参考资料

https://mp.weixin.qq.com/s/Aj_P5r0Q4hwmWi5tttZy6w

https://github.com/wepe/O2O-Coupon-Usage-Forecast

https://tianchi.aliyun.com/notebook/23504

https://github.com/jack12356/tianchi-o2o/blob/master/champion_EDA.ipynb

In [27]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# 导入时将缺失值统一记为 null，方便 比较
online = pd.read_csv('online_train.csv', header=0, keep_default_na=False)
offline = pd.read_csv('offline_train.csv', header=0, keep_default_na=False)

offline_test = pd.read_csv('offline_test.csv', header=0, keep_default_na=False)


#大写改小写，不然容易打错字
online.columns = ['user_id', 'merchant_id', 'action', 'coupon_id', 'discount_rate','date_received', 'date']
offline.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']
offline_test.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']

In [16]:
def merchant_processing (Dataset):
    
    merchant = Dataset[['merchant_id', 'coupon_id', 'distance', 'date_received', 'date']]

    m = merchant[['merchant_id']]
    m.drop_duplicates(inplace=True)

    m1 = merchant[merchant.date != 'null'][['merchant_id']]
    m1['total_sales'] = 1
    m1 = m1.groupby('merchant_id').agg('sum').reset_index()

    m2 = merchant[(merchant.date != 'null') & (merchant.coupon_id != 'null')][['merchant_id']]
    m2['sales_use_coupon'] = 1
    m2 = m2.groupby('merchant_id').agg('sum').reset_index()

    m3 = merchant[merchant.coupon_id != 'null'][['merchant_id']]
    m3['total_coupon'] = 1
    m3 = m3.groupby('merchant_id').agg('sum').reset_index()

    m4 = merchant[(merchant.date != 'null') & (merchant.coupon_id != 'null')][['merchant_id', 'distance']]
    m4.replace('null', -1, inplace=True)
    m4.distance = m4.distance.astype('int')
    m4.replace(-1, np.nan, inplace=True)
    
    m5 = m4.groupby('merchant_id').agg('min').reset_index()
    m5.rename(columns={'distance': 'merchant_distance_min'}, inplace=True)

    m6 = m4.groupby('merchant_id').agg('max').reset_index()
    m6.rename(columns={'distance': 'merchant_distance_max'}, inplace=True)

    m7 = m4.groupby('merchant_id').agg('mean').reset_index()
    m7.rename(columns={'distance': 'merchant_distance_mean'}, inplace=True)

    m8 = m4.groupby('merchant_id').agg('median').reset_index()
    m8.rename(columns={'distance': 'merchant_distance_median'}, inplace=True)

    merchant_feature = pd.merge(m, m1, on='merchant_id', how='left')
    merchant_feature = pd.merge(merchant_feature, m2, on='merchant_id', how='left')
    merchant_feature = pd.merge(merchant_feature, m3, on='merchant_id', how='left')
    merchant_feature = pd.merge(merchant_feature, m5, on='merchant_id', how='left')
    merchant_feature = pd.merge(merchant_feature, m6, on='merchant_id', how='left')
    merchant_feature = pd.merge(merchant_feature, m7, on='merchant_id', how='left')
    merchant_feature = pd.merge(merchant_feature, m8, on='merchant_id', how='left')
   
    merchant_feature.sales_use_coupon = merchant_feature.sales_use_coupon.replace(np.nan, 0)  #为 m2 填充缺失值为 0 ，方便 m8计算
    merchant_feature[
        'merchant_coupon_transfer_rate'] = merchant_feature.sales_use_coupon.astype(
            'float') / merchant_feature.total_coupon

    merchant_feature['coupon_rate'] = merchant_feature.sales_use_coupon.astype(
        'float') / merchant_feature.total_sales
    merchant_feature.total_coupon = merchant_feature.total_coupon.replace(np.nan, 0) #填充缺失值为 0
    
    merchant_feature.fillna('null',inplace=True) #更改缺失值为 null
    
    return merchant_feature

In [17]:
merchant_processing = merchant_processing (offline)
#merchant_processing.to_csv('merchant_processing.csv', index=None) #需要的话可以输出 csv

C:\Users\10096\AppData\Local\Temp\ipykernel_19300\2418528460.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m.drop_duplicates(inplace=True)


In [18]:
merchant_processing.head()

,merchant_id,total_sales,sales_use_coupon,total_coupon,merchant_distance_min,merchant_distance_max,merchant_distance_mean,merchant_distance_median,merchant_coupon_transfer_rate,coupon_rate
0,2632,21.0,4.0,43.0,0.0,1.0,0.75,1.0,0.093023,0.190476
1,4663,1434.0,76.0,15191.0,0.0,10.0,2.073529,1.0,0.005003,0.052999
2,3381,21843.0,2487.0,122834.0,0.0,10.0,1.652429,1.0,0.020247,0.113858
3,450,12285.0,1461.0,63042.0,0.0,10.0,0.892164,0.0,0.023175,0.118926
4,6459,20.0,0.0,16.0,null,null,null,null,0.0,0.0


In [24]:
#检查商户id是否重复
merchant_processing.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
8410    False
8411    False
8412    False
8413    False
8414    False
Length: 8415, dtype: bool